## Secondary Indexes in ver 6.1 for GeoJSON Data
 
 <img src="./graphics/SIOverview.png"
     alt="SIOverview"
     style="center; margin-right: 10px;"
     width="800"
     height="640"/>


#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>9.0.3</version>
  </dependency>
</dependencies>

####  Add required Java Client Imports

These are some of the Aerospike Java Client imports needed to start developing our Application interactively.  We will add others, as needed, as we develop our solution.

In [3]:
//Require Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


#### Connect to the Aerospike Server
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [4]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


### Geo2DSphere Sindex Test
#### Truncate previous records and generate geoJSON test records

In [5]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### GeoJSON Point, Polgon String generator - Helper functions

In [6]:
//Generate Point Format GeoJSON string

//Point: order of data is longitude, latitude 

String generatePoint(double longitude, double latitude){
  return String.format("{\"type\": \"Point\", \"coordinates\": [%f, %f]}", longitude, latitude);
}

//Four corner polygon.  Specify in counterclockwise order
String generatePolygon(double ax, double ay, double bx, double by, double cx, double cy, double dx, double dy){
  return String.format(
  "{\"type\": \"Polygon\", \"coordinates\": [[[%f, %f], [%f, %f],[%f, %f],[%f, %f],[%f, %f]]]}", 
  ax, ay,
  bx, by,
  cx, cy,
  dx, dy,
  ax, ay
  );
}

String generateRectangle( double ax, double ay, double length, double height)
{
   length = length/2;
   height = height/2;
   return generatePolygon(ax-length, ay-height, ax+length, ay-height, ax+length, ay+height, ax-length,ay+height);
}
String generateAeroCircle(double x, double y, int r){
  return String.format(
  "{\"type\": \"AeroCircle\", \"coordinates\": [[%.8f, %.8f], %d] }", x, y, r);
}

System.out.println(generatePoint(0.0,0.0));
System.out.println(generateRectangle(0.0,0.0,4.0, 2.0));
System.out.println(generateAeroCircle(0.0,0.0,1));


{"type": "Point", "coordinates": [0.000000, 0.000000]}
{"type": "Polygon", "coordinates": [[[-2.000000, -1.000000], [2.000000, -1.000000],[2.000000, 1.000000],[-2.000000, 1.000000],[-2.000000, -1.000000]]]}
{"type": "AeroCircle", "coordinates": [[0.00000000, 0.00000000], 1] }


## Import Modules
Let us start by importing all the modules we will need for the SI examples.

In [7]:
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.cdt.CTX;

In [8]:
final String Namespace = "test";
final String Set = "cdt-indexing";

// convenience function to create an index - essentially a pass-through to the client API
void createIndex(String idxName, String binName, IndexType idxType, IndexCollectionType colType, CTX... ctx) {
    try {
            IndexTask task = client.createIndex(null,
                                       Namespace,
                                       Set,
                                       idxName,
                                       binName,
                                       idxType,
                                       colType,
                                       ctx);
            task.waitTillComplete(1000, 0);
        }
        catch (AerospikeException ae) {
            if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
                throw ae;
            }
        } 
        System.out.format("Created index %s on ns=%s set=%s bin=%s.\n", 
                                    idxName, Namespace, Set, binName);
}

// convenience function to drop an index - essentially a pass-through to the client API
void dropIndex(String idxName) {
    try {
        IndexTask task = client.dropIndex(null, Namespace, Set, idxName);
    }
    catch (AerospikeException ae) {
        if (ae.getResultCode() != ResultCode.INDEX_NOTFOUND) {
            throw ae;
        }
    } 
    System.out.format("Dropped index %s.\n", idxName);
}


In [9]:
//Insert a record using a class based construct
final String geoLocationBinName = "location";
final String geoRegionBinName = "region";
class geoUser {
  public void createLocationRecord(String ns, String setName, String user, double x, double y, boolean bCheck) {
    Key userKey = new Key(ns, setName, user); 
    String sLocPoint = generatePoint(x, y);
    Bin bLoc = new Bin(geoLocationBinName, Value.getAsGeoJSON(sLocPoint));    
    WritePolicy wPolicyGeo = new WritePolicy();
    wPolicyGeo.sendKey = true;
    client.put(wPolicyGeo, userKey, bLoc);
    if(bCheck){
      System.out.println(client.get(null, userKey));
    }   
  }
  
    public void createRegionRecord(String ns, String setName, String user, 
                                   double x, double y, double l, double h,boolean bCheck) {
    Key userKey = new Key(ns, setName, user); 
    String sRegion = generateRectangle(x, y, l, h);
    Bin bReg = new Bin(geoRegionBinName, Value.getAsGeoJSON(sRegion));    
    WritePolicy wPolicyGeo = new WritePolicy();
    wPolicyGeo.sendKey = true;
    client.put(wPolicyGeo, userKey, bReg);
    if(bCheck){
      System.out.println(client.get(null, userKey));
    }   
  }
}

void executeGeoQueryAndPrintResults(Filter filter, String binName) {
    Statement stmt = new Statement();
    stmt.setNamespace(Namespace);
    stmt.setSetName(Set);
    stmt.setFilter(filter);
    stmt.setBinNames(binName);
    RecordSet rs = client.query(null, stmt);
    while (rs.next()) {
        Key key = rs.getKey();
        Record record = rs.getRecord();
        System.out.format("key=%s bins=%s\n", key.userKey, record.bins);
    }
    //System.out.println();
    rs.close();
}

In [10]:
createIndex("idx_geoLoc", "location", IndexType.GEO2DSPHERE, IndexCollectionType.DEFAULT);

Created index idx_geoLoc on ns=test set=cdt-indexing bin=location.


In [11]:
createIndex("idx_geoReg", "region", IndexType.GEO2DSPHERE, IndexCollectionType.DEFAULT);

Created index idx_geoReg on ns=test set=cdt-indexing bin=region.


In [12]:
geoUser ul = new geoUser();
ul.createLocationRecord(Namespace, Set, "user1", 0.0, 1.0, false);
ul.createLocationRecord(Namespace, Set, "user2", 1.0, 1.0, false);
ul.createLocationRecord(Namespace, Set, "user3", -1.0, 1.0, false);
ul.createLocationRecord(Namespace, Set, "user4", -1.0, -1.0, false);
ul.createLocationRecord(Namespace, Set, "user5", 0.0, -1.0, false);
ul.createLocationRecord(Namespace, Set, "user6", 1.0, -1.0, false);
ul.createLocationRecord(Namespace, Set, "user7", 1.0, 4.0, false);
ul.createLocationRecord(Namespace, Set, "user8", -1.0, 4.0, false);
ul.createLocationRecord(Namespace, Set, "user9", 0.0, 4.0, false);
ul.createLocationRecord(Namespace, Set, "user0", 0.0, -4.0, false);

//Add region bin also
ul.createRegionRecord(Namespace, Set, "user1", 0.0, 1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user2", 1.0, 1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user3", -1.0, 1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user4", -1.0, -1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user5", 0.0, -1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user6", 1.0, -1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user7", 1.0, 4.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user8", -1.0, 4.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user9", 0.0, 4.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user0", 0.0, -4.0, 0.25, 0.25, true);

(gen:2),(exp:475977332),(bins:(location:{"type": "Point", "coordinates": [0.000000, 1.000000]}),(region:{"type": "Polygon", "coordinates": [[[-0.125000, 0.875000], [0.125000, 0.875000],[0.125000, 1.125000],[-0.125000, 1.125000],[-0.125000, 0.875000]]]}))
(gen:2),(exp:475977332),(bins:(location:{"type": "Point", "coordinates": [1.000000, 1.000000]}),(region:{"type": "Polygon", "coordinates": [[[0.875000, 0.875000], [1.125000, 0.875000],[1.125000, 1.125000],[0.875000, 1.125000],[0.875000, 0.875000]]]}))
(gen:2),(exp:475977332),(bins:(location:{"type": "Point", "coordinates": [-1.000000, 1.000000]}),(region:{"type": "Polygon", "coordinates": [[[-1.125000, 0.875000], [-0.875000, 0.875000],[-0.875000, 1.125000],[-1.125000, 1.125000],[-1.125000, 0.875000]]]}))
(gen:2),(exp:475977332),(bins:(location:{"type": "Point", "coordinates": [-1.000000, -1.000000]}),(region:{"type": "Polygon", "coordinates": [[[-1.125000, -1.125000], [-0.875000, -1.125000],[-0.875000, -0.875000],[-1.125000, -0.875000]

#### GeoJSON SI Query. Find all Records with Point Bin where the Point lies within a geoJSON Rectangle


In [13]:
String geoRect = generateRectangle(0.0,0.0,1.5,2.5);
System.out.println(geoRect);
Filter filter = Filter.geoContains(geoLocationBinName, geoRect);
executeGeoQueryAndPrintResults(filter, geoLocationBinName);

{"type": "Polygon", "coordinates": [[[-0.750000, -1.250000], [0.750000, -1.250000],[0.750000, 1.250000],[-0.750000, 1.250000],[-0.750000, -1.250000]]]}
key=user1 bins={location={"type": "Point", "coordinates": [0.000000, 1.000000]}}
key=user5 bins={location={"type": "Point", "coordinates": [0.000000, -1.000000]}}


In [14]:
String geoPoint = generatePoint(1.0, -1.0);
System.out.println(geoPoint);
Filter filter = Filter.geoContains(geoRegionBinName, geoPoint);
executeGeoQueryAndPrintResults(filter, geoRegionBinName);

{"type": "Point", "coordinates": [1.000000, -1.000000]}
key=user6 bins={region={"type": "Polygon", "coordinates": [[[0.875000, -1.125000], [1.125000, -1.125000],[1.125000, -0.875000],[0.875000, -0.875000],[0.875000, -1.125000]]]}}


#### GeoJSON SI Query. Find all Records with Point Bin where the Point lies within a geoJSON AeroCircle
**Note**
1 deg latitude is 69 miles or 110,400 meters. So choose radius in meters accordingly. All coordinates are in longitude and latitude degree units in GeoJSON.

In [15]:
String geoCircle = generateAeroCircle(1.0,1.0,125000);
System.out.println(geoCircle);
//Filter filter = Filter.geoWithinRadius(geoLocationBinName, 0.0,0.0,125000);
//Filter filter = Filter.geoWithinRegion(geoLocationBinName, IndexCollectionType.DEFAULT, geoCircle);
//Filter filter = Filter.geoWithinRegion(geoLocationBinName, geoCircle);
Filter filter = Filter.geoContains(geoLocationBinName, geoCircle);
executeGeoQueryAndPrintResults(filter, geoLocationBinName);

{"type": "AeroCircle", "coordinates": [[1.00000000, 1.00000000], 125000] }
key=user2 bins={location={"type": "Point", "coordinates": [1.000000, 1.000000]}}
key=user1 bins={location={"type": "Point", "coordinates": [0.000000, 1.000000]}}


In [16]:

Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("cdt-indexing");
String location="{ \"type\": \"Point\", \"coordinates\": [0.0, 1.0] }";
Filter filter = Filter.geoContains("region", location);
stmt.setFilter(filter);
//stmt.setBinNames("location");

RecordSet rs = client.query(null, stmt);
while (rs.next()) {
    Key key = rs.getKey();
    Record record = rs.getRecord();
    System.out.format("key=%s bins=%s\n", key.userKey, record.bins);
}
//System.out.println();
rs.close();

key=user1 bins={location={"type": "Point", "coordinates": [0.000000, 1.000000]}, region={"type": "Polygon", "coordinates": [[[-0.125000, 0.875000], [0.125000, 0.875000],[0.125000, 1.125000],[-0.125000, 1.125000],[-0.125000, 0.875000]]]}}


## Clean up

In [17]:
dropIndex("idx_geoLoc");
dropIndex("idx_geoReg");

Dropped index idx_geoLoc.
Dropped index idx_geoReg.


In [18]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"